<a href="http://folk.ntnu.no/leifh/teaching/tkt4140/._main057.html" target="_blank">Iterative Methods</a>

N = 4 <br>
prototype

<h4>Item <i>"a)"</i> Onde f = 0 e u(x,y)= alpha * np.exp(x) * np.sin(y) na fronteira</h4>

In [43]:
import numpy as np
import pandas as pd
from time import time

<h3>Funções Auxiliares</h3>

<h4>Jacobi</h4>

In [44]:
def jacobi_method(U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i, j] = 0.25*(U_old[i-1, j] + U_old[i+1, j] + U_old[i, j-1] + U_old[i, j+1] + h**2 * F[i, j])   

<h4>Gauss-Seidel</h4>

In [45]:
def g_s_method(U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i, j] = 0.25*(U_new[i-1, j] + U_old[i+1, j] + U_new[i, j-1] + U_old[i, j+1] + h**2 * F[i, j])   

<h4>SOR</h4>

In [46]:
def sor_method(w, U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i,j] =  (w/4.0) * (U_new[i-1,j] + U_old[i+1,j] + U_new[i,j-1] + U_old[i,j+1] + h**2 *F[i,j]) + (1.-w)*U_old[i,j];


<h3>Funções Vetoriais Auxiliares</h3>

<h4>Jacobi</h4>

In [47]:
def jacobi_method_vector(U_old, U_new, h, F):
    
    left_points  = U_old[0:N-1, 1:N].copy()
    right_points = U_old[2:N+1, 1:N].copy()
    down_points  = U_old[1:N, 0:N-1].copy()
    up_points    = U_old[1:N, 2:N+1].copy()
        
    f_val = F[1:N, 1:N].copy()
    f_val = h**2 * f_val
    
    result = 0.25 * (left_points + right_points + down_points + up_points + f_val)

    U_new[1:N, 1:N] = result.copy()
        

<h4>PASSO 1 e 2</h4>

In [48]:
def initialize_matrixes():
    U_old = np.zeros((N+1,N+1))
    U_new = np.zeros((N+1,N+1))
    F = np.zeros((N+1,N+1)) 
    
    return U_old, U_new, F

<h4>PASSO 3</h4>

In [49]:
def initialize_U_answer():
    
    U_answer = np.zeros((N+1,N+1)) 

    for j_y in range(N+1):
        for i_x in range(N+1):
            U_answer[i_x, j_y] = alpha * np.exp(h*i_x) * np.sin(h*j_y)
    
    return U_answer

<h4>PASSO 4</h4>

In [50]:
def update_border(U_old, U_new):
    # Parte inferior e superior da Malha
    for i in range(N+1):
        U_old[i, 0] = alpha * np.exp(h*i) * np.sin(h*0.)
        U_old[i, N] = alpha * np.exp(h*i) * np.sin(h*N)

    # Parte esquerda e direita da Malha
    for j in range(N+1):
        U_old[0, j] = alpha * np.exp(h*0.) * np.sin(h*j)
        U_old[N, j] = alpha * np.exp(h*N) * np.sin(h*j)    
    
    #U_old e U_new têm o mesmo valor nas bordas
    U_new = U_old.copy()
    
    return U_new, U_old

In [51]:
def calculate_difference(U_new, U_old, N):
    
    quadratic_diff = (U_new[1:N-1, 1:N-1] - U_old[1:N-1, 1:N-1]) **2
    return h * np.sqrt((quadratic_diff.sum()))

<h4>PASSO 5-7</h4>

In [52]:
def call_g_s(U_old, U_new, h, F, TOL, U_answer, MAXITER):
    
    start = time()
    n_iterations = 0
    
    for it in range(80000):

        g_s_method(U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('Gauss-Seidel convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()
#     print('%f Segundos' %(end-start), '\nErro: ',np.amax(abs(U_answer - U_new)))
    return ['Gauss-Seidel', N, n_iterations, (end-start), np.amax(abs(U_answer - U_new))]


In [53]:
def call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER):
    
    start = time()
    n_iterations = 0

    for it in range(80000):
        jacobi_method(U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('Jacobi convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()
#     print('%f Segundos' %(end-start), '\nErro: ',np.amax(abs(U_answer - U_new)))    
    return ['Jacobi', N, n_iterations, (end-start), np.amax(abs(U_answer - U_new))]


In [54]:
def call_sor(U_old, U_new, h, F, TOL, U_answer, MAXITER):
    
    start = time()
    n_iterations = 0
    w = 2./(1. + np.sin(np.pi*h))
    
    for it in range(80000):
        sor_method(w, U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('SOR convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()
#     print('%f Segundos' %(end-start), '\nErro: ',np.amax(abs(U_answer - U_new)))    
    return ['SOR', N, n_iterations, (end-start), np.amax(abs(U_answer - U_new))]


In [55]:
df = pd.DataFrame({},columns=['Método', 'tamanho de N', 'num. iterações', 'tempo(seg)', 'erro absoluto máx'])

In [59]:
N=2**5
MAXITER=80000

alpha = 2.0
h=1./N

TOL = 10**-5 * h

U_old, U_new, F = initialize_matrixes()
U_answer = initialize_U_answer()
U_new, U_old = update_border(U_old, U_new)

call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER)
# call_g_s(U_old, U_new, h, F, TOL, U_answer, MAXITER)
# call_sor(U_old, U_new, h, F, TOL, U_answer, MAXITER)

#df.loc[-1] = call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER)
#df.index += 1

Jacobi convergiu com 2049 iterações


['Jacobi', 32, 2049, 3.5075416564941406, 0.00011004883299770007]